In [1]:
# Import Core Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Set Theme for Plotly Maps
# Different MapBox Styles
# carto-darkmatter
# open-street-map
# carto-positron

# Set Global Map Theme
map_theme = 'carto-darkmatter'

# Set Theme for graphs
sns.set_theme(palette='bright')

# Set Pandas Parameters
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)

# Import DataFrame
stl_df = pd.read_csv('../Data/STL-Data-2023.csv')

# Create County DataFrame
stl_county_df = stl_df.loc[stl_df['Geo-County'] == 'Saint Louis County']

# Create City DataFrame
stl_city_df = stl_df.loc[stl_df['Geo-County'] == 'Saint-Louis City']

In [48]:
shti = 'neights/'
new_shit = shti[:-1]
print(new_shit)

neights


## Directory Structure

In [52]:
def createAreaDirectories(dataFrame, areaColumn, areaPath):
    '''
    Title: Create Area Directories
    Description: This will create seperate directories for each area in areaColumn.
    dataFrame: The dataframe of data that the area columns are in.
    areaColumn: The specfic column of places that need directories created.
    area_path: The path in which you would like the directories created in.
    '''

    # Create list of all area names
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    
    bigDirExists = os.path.exists(areaPath)

    # Check if directory already exists
    if bigDirExists == False:
        # Create directory
        os.mkdir(areaPath)
    else:
        pass

    for area in area_list:

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Set full path to a variable
        fullPath = os.path.join(areaPath,area)

        # Directory boolean
        littleDirExists = os.path.exists(fullPath)

        # Check if directory already exists
        if littleDirExists == False:
            # Create directory
            os.mkdir(fullPath)
        else:
            pass

# Create a Folder for each Town
createAreaDirectories(dataFrame=stl_df,
                      areaColumn='Geo-Town',
                      areaPath='Towns/')

# Create a Folder for each Village
createAreaDirectories(dataFrame=stl_df,
                      areaColumn='Geo-Village',
                      areaPath='Villages/')

# Create a Folder for each Neighborhood
createAreaDirectories(dataFrame=stl_df,
                      areaColumn='Geo-Neighborhood',
                      areaPath='Neighborhoods/')

## Create Visualizations

### Area Pictures

In [53]:
def createAreaPictures(dataFrame, areaColumn, nibrsColumn, zoomLevel, areaPath):
    '''
    Title: Area Map Picture
    Description: This will create cover images to use in the website as backgrounds. 
    The colors are correlated to the amound of crimes and for each area.
    dataFrame: The dataframe in which all data is found. 
    areaCol: The column name of the area in which you would to create images for.
    nibrsCol: The column name of the type of nibrs information that would be mapped.
    imagePath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())

        # Create dictionary
        area_vc_mappings = dict()

        # Reverse the order of dataframe
        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsColumn],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsColumn].map(area_vc_mappings)
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']
                                                  
        # Create Geographic Map
        mapPlot = go.Figure(go.Scattermapbox(name=f'{areaColumn} Crime Points',   # Chart Name
                             lat=area_latitudes,                                  # Latitudes
                             lon=area_longitudes,                                 # Longitudes
                             showlegend=False,                                    # Don't show legends
                             marker=dict(color=area_df['NIBRS-Map'])))            # Hover Information Template     

        # Create Layout
        mapPlot.update_layout(height= 900,                                                # Height of Subplot
                              width= 1400,                                                # Width of Subplot
                              margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                              autosize=True,                                              # Set Autosize of charts to True or False
                              template='plotly_dark',                                     # Template of Subplot
                              mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                          pitch=0,                                        # Pitch of Map
                                          bearing=0,                                      # Bearing of Map
                                          style=map_theme,                                # Theme of Scatter Map
                                          center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                      lon=np.mean(area_longitudes))))     # Mean of Area Longitudes
        
        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")
        
        # print(area)

        coverImagePath = f'CoverImage-2023-{area}.jpeg'
        areaFullPath = f'{areaPath}{area}'

        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save image
        mapPlot.write_image(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Pictures Created for {areaColumn}!')

# Create Cover Images for each Town, Village & Neighborhood
createAreaPictures(dataFrame=stl_df,              # DataFrame
                   areaColumn='Geo-Town',         # Column of Area
                   nibrsColumn='NIBRS-Offense',   # NIBRS Crime Column
                   zoomLevel=12.5,                # Zoom Level        
                   areaPath='Towns/')             # Path to directory

createAreaPictures(dataFrame=stl_df,              # DataFrame
                   areaColumn='Geo-Village',      # Column of Area
                   nibrsColumn='NIBRS-Offense',   # NIBRS Crime Column
                   zoomLevel=12.5,                # Zoom Level    
                   areaPath='Villages/')          # Path to directory

createAreaPictures(dataFrame=stl_df,                # DataFrame
                   areaColumn='Geo-Neighborhood',   # Column of Area
                   nibrsColumn='NIBRS-Offense',     # NIBRS Crime Column
                   zoomLevel=14,                    # Zoom Level    
                   areaPath='Neighborhoods/')       # Path to directory  

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


### Density Maps

In [54]:
def createAreaDensityMap(dataFrame, areaColumn, zoomLevel, areaPath):
    '''
    Title: Create Area Denisty Map
    Description: This function will create Area Denisty Maps to show where 
    crime is most common, but correlating with the other crime around them.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    zoomLevel: The zoom number to see the mapped points.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']
                                                  
        # Create Density Plot
        densityPlot = go.Figure(go.Densitymapbox(name='Crime Density Map',   # Denisty Map Name             
                                                 lat=area_latitudes,         # Latitude values          
                                                 lon=area_longitudes,        # Longitude values       
                                                 radius=10))                 # Radius

        # Create Layout
        densityPlot.update_layout(height= 900,                                              # Height of Subplot
                                  width= 1400,                                                # Width of Subplot
                                  margin={"r":0,"t":0,"l":0,"b":0},                           # Margin values right, top, left, bottom                  
                                  autosize=True,                                              # Set Autosize of charts to True or False
                                  template='plotly_dark',                                     # Template of Subplot
                                  mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                              pitch=0,                                        # Pitch of Map
                                              bearing=0,                                      # Bearing of Map
                                              style=map_theme,                                # Theme of Scatter Map
                                              center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                          lon=np.mean(area_longitudes))))     # Mean of Area Longitudes

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Set Path Information
        coverImagePath = f'DensityMap-2023-{area}.html'
        areaFullPath = f'{areaPath}{area}'

        # Set final path to save
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save image
        densityPlot.write_html(finalPath)
    
    # Show successful creation
    print(f'[SUCCESS] Area Density Maps are Created for {areaColumn}!')

# Create & Save Density Maps
createAreaDensityMap(dataFrame=stl_df,        # DataFrame
                     areaColumn='Geo-Town',   # Column of Area
                     zoomLevel=12.5,          # Zoom Level        
                     areaPath='Towns/')       # Path to directory

createAreaDensityMap(dataFrame=stl_df,           # DataFrame
                     areaColumn='Geo-Village',   # Column of Area
                     zoomLevel=12.5,             # Zoom Level    
                     areaPath='Villages/')       # Path to directory

createAreaDensityMap(dataFrame=stl_df,                # DataFrame
                     areaColumn='Geo-Neighborhood',   # Column of Area
                     zoomLevel=14,                    # Zoom Level    
                     areaPath='Neighborhoods/')       # Path to directory  

[SUCCESS] Area Density Maps are Created for Geo-Town!
[SUCCESS] Area Density Maps are Created for Geo-Village!
[SUCCESS] Area Density Maps are Created for Geo-Neighborhood!


### Scatter Maps

In [ ]:
# Show all color swatches for continuous colors
fig = px.colors.sequential.swatches_continuous() 
fig.show() 

In [56]:
def createAreaScatterMap(dataFrame, areaColumn, nibrsColumn, zoomLevel, areaPath):
    '''
    Title: Create Area Scatter Map
    Description: This function creates a scatter map of all the latitudes and longitudes
    correlating to the color of how many crime points there are.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    zoomLevel: The zoom number to see the mapped points.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())
        area_vc_mappings = dict()
        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsColumn],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsColumn].map(area_vc_mappings)

        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']

        # hovertempletes
        area_hover = areaColumn + ':' + '%{customdata[0]}'
        nibrs_hover = nibrsColumn + ':' + '%{customdata[1]}'

        # Create Scatter Map Base Plot
        scatterMap = go.Figure(go.Scattermapbox())

        # Create list of Colors that correlate to continuous color values like other charts
        markerColors = px.colors.sample_colorscale('plasma',                                             # Color palette list
                                                   samplepoints=area_vc['count']/area_vc['count'].max())   # The range of point from min-max

        # Create Counter
        i=0

        # Add each unique NIBRS column value to the scatter map in correlating colors
        for nibrsName in area_vc[nibrsColumn].unique():
            
            # Create that nibrs column value as it owns dataframe
            nibs_df = area_df[area_df[nibrsColumn] == nibrsName]
            
            # Create each trace of Points
            scatterMap.add_trace(go.Scattermapbox(visible=True,                                             # Make sure all points are visible
                                                  name=nibrsName,                                           # Value of NIBRS for legend corrlation
                                                  mode="markers",                                           # Set mode to markers
                                                  lat=nibs_df['Geo-Latitude'],                              # Latitude Points
                                                  lon=nibs_df['Geo-Longitude'],                             # Longitude Points
                                                  marker=dict(color=f'{markerColors[i]}'),                  # Set the color of each point
                                                  customdata=nibs_df[[areaColumn, nibrsColumn]],            # Custom Data Columns for Hover
                                                  hovertemplate="<br>".join([area_hover, nibrs_hover])))    # Hover value structure
            
            # Add to counter
            i = i+1

        # Update ScatterMap Layout
        scatterMap.update_layout(height= 900,                                                   # Height of Subplot
                                 width= 1400,                                                   # Width of Subplot
                                 margin={"r":0,"t":0,"l":0,"b":0},                              # Margin values right, top, left, bottom                  
                                 autosize=True,                                                 # Set Autosize of charts to True or False
                                 template='plotly_dark',                                        # Template of Subplot
                                 legend=dict(title=f'{nibrsColumn}'),                           # Legend title
                                 mapbox=dict(zoom=zoomLevel,                                    # Zoom of Map
                                                pitch=0,                                        # Pitch of Map
                                                bearing=0,                                      # Bearing of Map
                                                style=map_theme,                                # Theme of Scatter Map
                                                center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                            lon=np.mean(area_longitudes))))     # Mean of Area Longitudes    # Mean of Area Longitudes
        
        # Add Geographical Data Quality Annotation
        scatterMap.add_annotation(dict(y=0.01,                                             # y position 
                                       x=0.01,                                             # X position 
                                       yref="paper",                                       # set y reference to paper
                                       xref="paper",                                       # Set X reference to paper
                                       xanchor='left',                                     # Set x anchor
                                       showarrow=False,                                    # Don't show arrow
                                       textangle=0,                                        # Angle of Title
                                       font=dict(color='grey',                             # Font color
                                                 size=8),                                  # Font size
                                       text=f"Geographical Data has accuracy of 99%"))     # Title of Annotation
    

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'ScatterMap-2023-({nibrsColumn})-{area}.html'
        areaFullPath = f'{areaPath}{area}'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        scatterMap.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Scatter Maps are Created for {areaColumn}!')

# Create Scatter Maps
createAreaScatterMap(dataFrame=stl_df,              # DataFrame
                     areaColumn='Geo-Town',         # Area Column
                     nibrsColumn='NIBRS-Offense',   # NIBRS Column
                     zoomLevel=12.5,                # Zoom Level value
                     areaPath='Towns/')             # Directory location

createAreaScatterMap(dataFrame=stl_df,              # DataFrame        
                     areaColumn='Geo-Village',      # Area Column        
                     nibrsColumn='NIBRS-Offense',   # NIBRS Column             
                     zoomLevel=12.5,                # Zoom Level value               
                     areaPath='Villages/')          # Directory location           

createAreaScatterMap(dataFrame=stl_df,                # DataFrame          
                     areaColumn='Geo-Neighborhood',   # Area Column                  
                     nibrsColumn='NIBRS-Offense',     # NIBRS Column        
                     zoomLevel=14,                    # Zoom Level value      
                     areaPath='Neighborhoods/')       # Directory location                     

[SUCCESS] Area Scatter Maps are Created for Geo-Town!
[SUCCESS] Area Scatter Maps are Created for Geo-Village!
[SUCCESS] Area Scatter Maps are Created for Geo-Neighborhood!


### BarCharts

In [55]:
def createAreaBarChart(dataFrame, areaColumn, nibrsColumn, areaPath):
    '''
    Title: Create Bar Chart
    Description: This function creates a barchart of all the crime for that given area
    with the following NIBRS column.
    dataFrame: The dataframe in which all data is found. 
    areaColumn: The column name of the area in which you would to create images for.
    areaPath: Path to where these images will be saved.
    '''

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())

        # Create Bar Chart
        barChart = go.Figure(go.Bar(name=f'Total Crime by {nibrsColumn}',   # Chart Title
                             text=area_vc["count"],                         # Add Data Labels to bars
                             orientation='h',                               # Orientation of Chart
                             y=area_vc[nibrsColumn],                        # y Values
                             x=area_vc['count'],                            # x values
                             showlegend=False,                              # Do not show legend
                             marker=dict(color=area_vc['count'])))          # Continuous colors based on amount

        # Update BarChart Layout
        barChart.update_layout(height= 900,                                                 # Height of Subplot
                               width= 1400,                                                 # Width of Subplot
                               margin={"r":0,"t":70,"l":0,"b":75},                          # Margin values right, top, left, bottom    
                               title=dict(text=f'2023 {area} {nibrsColumn} Crime Totals',   # Title of BarChart
                                          font=dict(size=30)),                              # Title Font size
                               autosize=True,                                               # Set Autosize of charts to True or False
                               template='plotly_dark',                                      # Template of Subplot
                               yaxis=dict(autorange="reversed",                             # Reverse Y-axis order
                                          title=dict(text=f'{nibrsColumn}',                 # Y-axis title
                                                     font=dict(size=20))),                  # Y-axis title font size
                               xaxis=dict(title=dict(text=f'Amount',                        # X-axis title
                                                     font=dict(size=20))))                  # X-axis title font size
        
        # Add Totla Crime Stats
        barChart.add_annotation(dict(y=1.05,                                    # y position 
                                     x=0.78,                                    # X position 
                                     yref="paper",                              # set y reference to paper
                                     xref="paper",                              # Set X reference to paper
                                     xanchor='left',                            # Set x anchor
                                     showarrow=False,                           # Don't show arrow
                                     textangle=0,                               # Angle of Title
                                     font=dict(color='white',                   # Font color
                                               size=20),                        # Font size
                                     text=f"Total Crimes: {len(area_df)}"))     # Title of Annotation

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'BarChart-2023-{nibrsColumn})-{area}.html'
        areaFullPath = f'{areaPath}{area}'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        barChart.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Scatter Maps are Created for {areaColumn}!')

# Create BarCharts
createAreaBarChart(dataFrame=stl_df,              # DataFrame
                   areaColumn='Geo-Town',         # Area Column           
                   nibrsColumn='NIBRS-Offense',   # NIBRS Column                  
                   areaPath='Towns/')             # Directory Path

createAreaBarChart(dataFrame=stl_df,              # DataFrame               
                   areaColumn='Geo-Village',      # Area Column                  
                   nibrsColumn='NIBRS-Offense',   # NIBRS Column                       
                   areaPath='Villages/')          # Directory Path            

createAreaBarChart(dataFrame=stl_df,                # DataFrame         
                   areaColumn='Geo-Neighborhood',   # Area Column                           
                   nibrsColumn='NIBRS-Offense',     # NIBRS Column                     
                   areaPath='Neighborhoods/')       # Directory Path                  

[SUCCESS] Area Scatter Maps are Created for Geo-Town!
[SUCCESS] Area Scatter Maps are Created for Geo-Village!
[SUCCESS] Area Scatter Maps are Created for Geo-Neighborhood!


### Scatter Maps & BarChart Combo

In [57]:
def createAreaScatterBarCombo(dataFrame, areaColumn, nibrsColumn, zoomLevel, areaPath):

    # Create list of all NeighborHoods
    area_list = dataFrame[dataFrame[areaColumn].notnull()][areaColumn].unique()

    # For each Neighborhood in the list
    for area in area_list:

        # Create DataFrame of Neighborhood
        area_df = dataFrame[dataFrame[areaColumn] == area].copy()

        # Value Count Lists
        area_vc = pd.DataFrame(area_df[nibrsColumn].value_counts().reset_index())

        area_vc_mappings = dict()

        area_vc_reversed = area_vc.iloc[::-1].copy()

        # Create Area NIBRS mappings
        for i, (offense, counts) in enumerate(zip(area_vc_reversed[nibrsColumn],area_vc_reversed['count'])):
            area_vc_mappings.update({f"{offense}": counts})

        # Map this counts to values in the area_df
        area_df['NIBRS-Map'] = area_df[nibrsColumn].map(area_vc_mappings)
            
        # Geographic Lists
        area_latitudes = area_df['Geo-Latitude']
        area_longitudes = area_df['Geo-Longitude']

        # hovertempletes
        area_hover = areaColumn + ':' + '%{customdata[0]}'
        nibrs_hover = nibrsColumn + ':' + '%{customdata[1]}'
        
        # Create Subplot Layout
        scatterBarCombo = make_subplots(rows=1,                                                # Number of Rows
                                     cols=2,                                                   # Number of columns
                                     horizontal_spacing=.28,                                   # Horizontal Spacing between columns
                                     column_widths=[0.5, 0.5],                                 # Column Widths max = 1.0
                                     specs=[[{"type": "scattermapbox"},{"type": "bar"}]])      # Specify Chart Types
                                                  
        # Create Geographic Map
        scatterBarCombo.add_scattermapbox(row=1,                                                 # Row in subplot
                                       col=1,                                                    # Column in subplot
                                       name=f'{areaColumn} Crime Points',                        # Chart Name
                                       lat=area_latitudes,                                       # Latitudes
                                       lon=area_longitudes,                                      # Longitudes
                                       showlegend=False,                                         # Do not show legend 
                                       marker=dict(color=area_df['NIBRS-Map']),                  # Set Markder color for each point
                                       customdata=area_df[[areaColumn, nibrsColumn]],            # Custom Data Columns for Hover
                                       hovertemplate="<br>".join([area_hover, nibrs_hover]))     # Hover Information Template     

        # Create Bar Chart
        scatterBarCombo.add_bar(row=1,                                                                # Row in Subplot
                             col=2,                                                                   # Column in subplot
                             name=f'Total Crime by {nibrsColumn}',                                    # Chart Title
                             text=area_vc["count"],                                                   # Add Data Labels to bars
                             orientation='h',                                                         # Orientation of Chart
                             y=area_vc[nibrsColumn],                                                  # y Values
                             x=area_vc['count'],                                                      # x values
                             showlegend=False,                                                        # Do not show legent
                             marker=dict(color=area_vc['count']),                                     # Marker Color
                             customdata=area_vc[['count',nibrsColumn]],                               # Custom Data Columns for Hover
                             hovertemplate="<br>".join([nibrs_hover,"Amount: %{customdata[0]}"]))     # Hover Information Template

        # Reverse order of values for Y-axis
        scatterBarCombo.update_yaxes(row=1,                 # Row in Subplot
                                  col=2,                    # Column in Subplot
                                  autorange="reversed")     # Reverse order of bars

        # Create Layout for Subplot
        scatterBarCombo.update_layout(height= 900,                                             # Height of Subplot
                                   width= 1400,                                                # Width of Subplot
                                   margin={"r":20,"t":60,"l":20,"b":10},                       # Margin values right, top, left, bottom                  
                                   autosize=True,                                              # Set Autosize of charts to True or False
                                   template='plotly_dark',                                     # Template of Subplot
                                   title=dict(text=f'{area} {nibrsColumn} Overview',           # Title of Subplot
                                              font=dict(color="White",                         # Font color
                                                        size=25)),                             # Font size
                                   font=dict(color='White',                                    # Font Color
                                             size=15),                                         # Font Size
                                   mapbox=dict(zoom=zoomLevel,                                 # Zoom of Map
                                               pitch=0,                                        # Pitch of Map
                                               bearing=0,                                      # Bearing of Map
                                               style=map_theme,                                # Theme of Scatter Map
                                               center=dict(lat=np.mean(area_latitudes),        # Mean of Area Latitudes
                                                           lon=np.mean(area_longitudes))))     # Mean of Area Longitudes

        # Add Geographical Data Quality Annotation
        scatterBarCombo.add_annotation(dict(y=0.01,                                          # y position 
                                         x=0.01,                                             # X position 
                                         yref="paper",                                       # set y reference to paper
                                         xref="paper",                                       # Set X reference to paper
                                         xanchor='left',                                     # Set x anchor
                                         showarrow=False,                                    # Don't show arrow
                                         textangle=0,                                        # Angle of Title
                                         font=dict(color='grey',size=8),                     # Font color, size
                                         text=f"Geographical Data has accuracy of 99%"))     # Title of Annotation
        
        # Add Totla Crime Stat
        scatterBarCombo.add_annotation(dict(y=1.05,                                 # y position 
                                         x=0.75,                                    # X position 
                                         yref="paper",                              # set y reference to paper
                                         xref="paper",                              # Set X reference to paper
                                         xanchor='left',                            # Set x anchor
                                         showarrow=False,                           # Don't show arrow
                                         textangle=0,                               # Angle of Title
                                         font=dict(color='white',                   # Font color
                                                   size=20),                        # Font size
                                         text=f"Total Crimes: {len(area_df)}"))     # Title of Annotation

        # Replace / or " " with a dash
        area = area.replace(" ", "-")
        area = area.replace("/","-")

        # Create Path information
        coverImagePath = f'ScatterMapBarChartCombo-2023-({nibrsColumn})-{area}.html'
        areaFullPath = f'{areaPath}{area}'
        
        # Set the full path
        finalPath = os.path.join(areaFullPath,coverImagePath)

        # Save HTML
        scatterBarCombo.write_html(finalPath)

    # Show successful creation
    print(f'[SUCCESS] Area Scatter Maps are Created for {areaColumn}!')
        
# Create Scatter Map & Barchart Combos
createAreaScatterBarCombo(dataFrame=stl_df,              # DataFrame                
                          areaColumn='Geo-Town',         # Area Column                  
                          nibrsColumn='NIBRS-Offense',   # NIBRS Column                       
                          zoomLevel=12,                  # Zoom Level value        
                          areaPath='Towns/')             # Directory location         

createAreaScatterBarCombo(dataFrame=stl_df,              # DataFrame                   
                          areaColumn='Geo-Village',      # Area Column                     
                          nibrsColumn='NIBRS-Offense',   # NIBRS Column                          
                          zoomLevel=12,                  # Zoom Level value             
                          areaPath='Villages/')          # Directory location              

createAreaScatterBarCombo(dataFrame=stl_df,                # DataFrame                   
                          areaColumn='Geo-Neighborhood',   # Area Column                              
                          nibrsColumn='NIBRS-Offense',     # NIBRS Column                          
                          zoomLevel=13.5,                  # Zoom Level value           
                          areaPath='Neighborhoods/')       # Directory location                 

[SUCCESS] Area Scatter Maps are Created for Geo-Town!
[SUCCESS] Area Scatter Maps are Created for Geo-Village!
[SUCCESS] Area Scatter Maps are Created for Geo-Neighborhood!


## Create HTML Source code

In [58]:
def stlWebLinks(areaCol, dataFrame, folder_name):
    # Create pandas DF to store names and sort them into alphabetical order
    areaNames = pd.DataFrame(columns=['Name'],data=dataFrame[dataFrame[areaCol].notnull()][areaCol].unique()).sort_values('Name').reset_index(drop=True)
    areaNames['SpaceFixed'] = areaNames['Name'].str.replace(" ", "-")
    areaNames['TotalFixed'] = areaNames['SpaceFixed'].str.replace("/", "-")

    for i in range(0,len(areaNames)):
        # Open file and set mode to append
        file1 = open("myfile.txt", "a")
    
        # writing newline character
        file1.write(f'''\n<!-- {areaNames['Name'][i]} -->
<a href="google.com">
    <div class="stl-port-box">
        <img src="{folder_name}/{areaNames['TotalFixed'][i]}.jpeg" alt="{areaNames["Name"][i]} Crime Points">
        <div class="stl-port-layer">
            <h4>{areaNames['Name'][i]}</h4>
        </div>
    </div>
</a>''')
    # Close and save file
    file1.close()

stlWebLinks('Geo-Neighborhood', stl_df, 'Neighborhoods')

                Name        SpaceFixed        TotalFixed
0            Academy           Academy           Academy
1              Baden             Baden             Baden
2        Benton Park       Benton-Park       Benton-Park
3   Benton Park West  Benton-Park-West  Benton-Park-West
4          Bevo Mill         Bevo-Mill         Bevo-Mill
..               ...               ...               ...
73  Walnut Park East  Walnut-Park-East  Walnut-Park-East
74  Walnut Park West  Walnut-Park-West  Walnut-Park-West
75  Wells Goodfellow  Wells-Goodfellow  Wells-Goodfellow
76          West End          West-End          West-End
77    Wydown Skinker    Wydown-Skinker    Wydown-Skinker

[78 rows x 3 columns]
